# Functions/Classes

In [53]:
"""
Persian Preprocessing tools:
https://github.com/ICTRC/Parsivar
https://github.com/Dadmatech
https://github.com/sobhe/hazm/
https://github.com/mhbashari/awesome-persian-nlp-ir/blob/master/sections/tools.md
"""

'\nPersian Preprocessing tools:\nhttps://github.com/ICTRC/Parsivar\nhttps://github.com/Dadmatech\nhttps://github.com/sobhe/hazm/\nhttps://github.com/mhbashari/awesome-persian-nlp-ir/blob/master/sections/tools.md\n'

In [54]:
# get rid of spacy and useless outputs 
from IPython.display import clear_output as cls

# for FarsiParsi
!pip install hazm 
!pip install parsivar
#!pip install dadmatools 

import hazm
import parsivar
#import dadmatools

# For reading csv files
import pandas as pd 

# For finding minimum edit distance
!pip install weighted-levenshtein
import numpy as np
from weighted_levenshtein import lev, osa, dam_lev

cls()

In [55]:
#import hazm
import re
from string import punctuation

class FarsiParsi:
  # Farsi Parser :) 
  def __init__(self,):
    self.normalize_func = hazm.Normalizer(affix_spacing=False,
                                          persian_numbers=False,
                                          persian_style=False).normalize
    self.word_tokenizer = hazm.word_tokenize
    st_words = hazm.stopwords_list()
    st_words.append('ی');st_words.append('های')
    self.stops = {}
    for k in st_words:
      self.stops[k] = False


  def clean_tokenize(self,string):
    d ='0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
    norm_txt = self.normalize_func(string)
    norm_txt = re.sub(f'[{d}{punctuation}؟،٪×÷»«؛]+', '', norm_txt)
    tokens = hazm.word_tokenize(norm_txt)
    res = [t for t in tokens if self.stops.get(t, True) and len(t)>2]
    return res
  

In [56]:
# import numpy as np
# from weighted_levenshtein import lev, osa, dam_lev
# Ref: https://pypi.org/project/weighted-levenshtein/
# Ref: https://www.geeksforgeeks.org/edit-distance-dp-5/

class SpellCorrection:

  def __init__(self, insert_cost= 1 ,delete_cost = 1,
               substitute_cost = 2, ) -> None:
      in_costs = np.ones(128, dtype=np.float64)
      dl_costs = np.ones(128, dtype=np.float64)  
      sb_costs = np.ones((128, 128), dtype=np.float64)  
      in_costs[in_costs > -1] = insert_cost; self.in_costs = in_costs
      dl_costs[dl_costs > -1] = delete_cost; self.dl_costs = dl_costs
      sb_costs[sb_costs > -1] = substitute_cost; self.sb_costs = sb_costs

  def _editDistance(self,str1, str2, m, n):
    if m == 0:
      return n

    # If second string is empty, the only option is to
    # remove all characters of first string
    if n == 0:
      return m

    # If last characters of two strings are same, nothing
    # much to do. Ignore last characters and get count for
    # remaining strings.
    if str1[m-1] == str2[n-1]:
      return self._editDistance(str1, str2, m-1, n-1)

    # If last characters are not same, consider all three
    # operations on last character of first string, recursively
    # compute minimum cost for all three operations and take
    # minimum of three values.
    return 1 + min(self._editDistance(str1, str2, m, n-1) + 0, # Insert
        self._editDistance(str1, str2, m-1, n) + 0, # Remove
        self._editDistance(str1, str2, m-1, n-1)+ 1  # Replace
        )
 
  def predict_from_scratch(self,string, top_bests = 5):
    # Keep Track of Top Best -> we need "Fixed-size Max Heap"
    all = []
    x = self._transliterate(string)
    m = len(x)
    for i in range(self.vocab_len):
      y = self.vocab_eng[i]
      n = len(y)
      c = self._editDistance(x, y, m, n)
      all.append( [c,self.vocab[i]] )

    res = sorted(all, key = lambda x: x[0], reverse= False)
    return res[:top_bests]

  def _transliterate(self, string):
    # encode persian to english (ASCII code)
    res = ''
    for s in string:
      res += self.per_eng_dict[s]
    return res

  def fit(self, per_eng_dict,vocab_lst):
    # Persian_to_English_dictionary is our decoding system
    self.vocab = vocab_lst
    self.vocab_len = len(vocab_lst)
    self.per_eng_dict = per_eng_dict
    # calculate correspoding transliteration for each token in vocab list
    vocab_eng = []
    for tok in self.vocab:
      vocab_eng.append(self._transliterate(tok))
    self.vocab_eng = vocab_eng


  def predict(self,string, top_bests = 5):
    # Keep Track of Top Best -> we need "Fixed-size Max Heap"
    all = []
    x = self._transliterate(string)
    for i in range(self.vocab_len):
      y = self.vocab_eng[i]
      c = lev(x, y, insert_costs=self.in_costs,
              delete_costs=self.dl_costs, substitute_costs=self.sb_costs)
      all.append( [c,self.vocab[i]] )

    res = sorted(all, key = lambda x: x[0], reverse= False)
    return res[:top_bests]


In [57]:
def accum_ngram(corp , n_gram = 5):
    
    # Python dictionary has dynamic hashing and can access each key's value 
    # in O(1) time. It, however, needs O(n) space to store n strings.  
    # "Trie" is an attempt to reduce this space complexity
    
    # accumulated n-grams 
    # by accumulated I mean we compute all n-grams from 1 up to n
    # Result will be stored and returned as a trie, implemented with dictionary
    
    n_grams = {}
    n_grams['#'] = 0 
    for stc in corp:
        n = len(stc)
        n_grams['#'] += n # update total words count
        for token_idx in range(n):
            dict_ptr = n_grams
            for next_token_idx in range(token_idx,n):
                if next_token_idx - token_idx >= n_gram:
                    break
                if not stc[next_token_idx] in dict_ptr:
                    dict_ptr[stc[next_token_idx]] = {'#':0}
                    
                dict_ptr[stc[next_token_idx]]['#'] += 1
                dict_ptr = dict_ptr[stc[next_token_idx]]
                
            
    return n_grams


# Here (in $$$ tagged line) I am using encouraged EAFP style
# https://docs.python.org/3.6/glossary.html#term-eafp
# https://stackoverflow.com/questions/1835756/using-try-vs-if-in-python

def count_finder(sentence, n_grams_dict, j, i,mu,k):
    # find both numerator and denominator 
    numerat ,denominat= k,mu
    ptr_dict = n_grams_dict
    flag = 1
    for id in range(j,i):
      # $$$
      try: 
        ptr_dict = ptr_dict[sentence[id]]
      except:
        # if we didn't have a key = sentence[id] in our tire
        # it means we don't have both numerator and denominator n_grams 
        flag = 0
        break

    if flag:
      denominat += ptr_dict['#']

    # Even if we have the n-gram associated with denominator
    # chances are that we don't have n-gram associated with numerator
    # since numerator's n-gram is longer that denominator's
    # following try-except statement is to check this
    try:
      tmp = ptr_dict[sentence[i]]['#']
      numerat += tmp
    except:
      pass

    return numerat ,denominat

def occurance_drichlet_probability(sentence, n_grams_dict, n_gram, mu,Pbg):
  # n_gram: numeber of previous words each conditional probability depends on
  # sentence: list of tokens

  # mu = k * v , Pbg= 1/v
  k = mu* Pbg
  
  # logic for finding uni-grams is slightly different than other grams
  # that's why we are handling uni-gram seperately
  if n_gram ==1:
      prob = 1
      for token in sentence:
          prob *= (n_grams_dict[token]['#']+k)
      prob *= (1/((n_grams_dict['#']+mu)**len(sentence)))
      return prob

  # here we handle x-grams: x>=2    
  # handeling the first fraction separately
  # # put the following line into try except, later :) 
  # prob = (n_grams_dict[sentence[0]]['#']+k)/(n_grams_dict['#']+mu) 
  first_num = 0
  first_deno = 0
  try:
    prob = (n_grams_dict[sentence[0]]['#']+k)/(n_grams_dict['#']+mu)
  except:
    prob = k/mu

  
  for i in range(1,len(sentence)):
      j = i - (n_gram -1) if (i-n_gram>=0) else 0
      numerator, denominator =count_finder(sentence,n_grams_dict,j,i,mu,k)
      prob *= (numerator/denominator)
      
  return prob

In [58]:
def question6_handler(candidate_words,bigrams_sent,accum_bigrams, mu,Pbg):
  i = -1
  for sent in bigrams_sent:
    i+=1
    best_w, best_prob = "", 0
    s = sent[1].split()

    for cand in candidate_words[i]:
      s[sent[0]] = cand
      p = occurance_drichlet_probability(s, accum_bigrams,n_gram=2, 
                                         mu = mu, Pbg = Pbg)
      if p > best_prob:
        best_prob = p
        best_w = cand
    print(f'Based on Bigrams, best word:{best_w} with prob:{best_prob}\n')

# Driver code

## Inputs

In [59]:
sent = [
        'انتخابات هفته آینده رادر تهران برگزار میکنیم ، اما دسته بندی حوزهها برای رأی گیری از این هفته آغاز شده است',
        'توانتشارخبرمهم در رسانه ها را تایید کردی',
        'پسردکتراحمدی درکارنامه حرفه ای خود ۱۸ سریال تلویزیونی و ۱۲ فیلم سینمایی دارد',
        'نقّاشی ساختمان به پایان رسیدددددددد',
        'اللة اکبر گویان وارد مسجد شد',
        'دکتر جان مممممممممممممممنونم که مشکلات حرکتي فرزندم را درمان کردید',
]

words = [
        'اختصاد',
        'سادرات',
        'فوتکال',
        'مسابغاط',
        'وازدات',
        'مشرکت',
        'کشوور',
        'منجلسه',
]


bigrams_sent = [
                 (1 ,"رشد اختصاد و تحرك زندگی اجتماعی"),
                 (1, 'حجم سادرات ایران'),
                 (1, 'فدراسیون فوتکال کشور'),
                 (3 ,'در جریان انعکاس مسابغاط صبح'),
                 (2, 'اقلام عمده وازدات کشور'),
                 (1 ,'اصل مشرکت مردمی'),
                 (1, 'وزارت کشوور جمهوری اسلامی ایران'),
                 (3 ,'جلسه علنی دیروز منجلسه شورای اسلامی'),
]

## Q1 (Norm, tok) 

In [60]:
!gdown --id 19n8IMFUzjHFrDe7N-AkVHpyBUeuuXory
!gdown --id 1Q1DZWyGv0HsNr4fwDxgBMl92P3Ntlq0B
cls()

In [61]:
"""  ----------------- Hazm ---------------------- """
hz_tokenizer = hazm.word_tokenize
hz_normalizer = hazm.Normalizer().normalize
for s in sent:
  print(hz_tokenizer(s))
  print(hz_normalizer(s))
  print()


['انتخابات', 'هفته', 'آینده', 'رادر', 'تهران', 'برگزار', 'میکنیم', '،', 'اما', 'دسته', 'بندی', 'حوزهها', 'برای', 'رأی', 'گیری', 'از', 'این', 'هفته', 'آغاز', 'شده_است']
انتخابات هفته آینده رادر تهران برگزار میکنیم، اما دسته بندی حوزهها برای رأی گیری از این هفته آغاز شده است

['توانتشارخبرمهم', 'در', 'رسانه', 'ها', 'را', 'تایید', 'کردی']
توانتشارخبرمهم در رسانه‌ها را تایید کردی

['پسردکتراحمدی', 'درکارنامه', 'حرفه', 'ای', 'خود', '۱۸', 'سریال', 'تلویزیونی', 'و', '۱۲', 'فیلم', 'سینمایی', 'دارد']
پسردکتراحمدی درکارنامه حرفه‌ای خود ۱۸ سریال تلویزیونی و ۱۲ فیلم سینمایی دارد

['نقّاشی', 'ساختمان', 'به', 'پایان', 'رسیدددددددد']
نقاشی ساختمان به پایان رسیدددددددد

['اللة', 'اکبر', 'گویان', 'وارد', 'مسجد', 'شد']
اللة اکبر گویان وارد مسجد شد

['دکتر', 'جان', 'مممممممممممممممنونم', 'که', 'مشکلات', 'حرکتي', 'فرزندم', 'را', 'درمان', 'کردید']
دکتر جان مممممممممممممممنونم که مشکلات حرکتی فرزندم را درمان کردید



In [62]:
print("""  ----------------- Hazm ---------------------- """,
      '\n','Normaliz then tokenize\n\n')
hz_tokenizer = hazm.word_tokenize
hz_normalizer = hazm.Normalizer().normalize
for s in sent:
  x = hz_tokenizer(hz_normalizer(s))
  for tok in x:
    print(tok)
  print('\nNext sentence')

  ----------------- Hazm ----------------------  
 Normaliz then tokenize


انتخابات
هفته
آینده
رادر
تهران
برگزار
میکنیم
،
اما
دسته
بندی
حوزهها
برای
رأی
گیری
از
این
هفته
آغاز
شده_است

Next sentence
توانتشارخبرمهم
در
رسانه‌ها
را
تایید
کردی

Next sentence
پسردکتراحمدی
درکارنامه
حرفه‌ای
خود
۱۸
سریال
تلویزیونی
و
۱۲
فیلم
سینمایی
دارد

Next sentence
نقاشی
ساختمان
به
پایان
رسیدددددددد

Next sentence
اللة
اکبر
گویان
وارد
مسجد
شد

Next sentence
دکتر
جان
مممممممممممممممنونم
که
مشکلات
حرکتی
فرزندم
را
درمان
کردید

Next sentence


In [63]:
"""  ----------------- Parsivar ---------------------- """
ps_tokenizer = parsivar.Tokenizer().tokenize_words
ps_normalizer = parsivar.Normalizer().normalize
for s in sent:
  print(ps_tokenizer(doc_string = s))
  print(ps_normalizer(doc_string = s))
  print()

['انتخابات', 'هفته', 'آینده', 'رادر', 'تهران', 'برگزار', 'میکنیم', '،', 'اما', 'دسته', 'بندی', 'حوزهها', 'برای', 'رأی', 'گیری', 'از', 'این', 'هفته', 'آغاز', 'شده', 'است']
انتخابات هفته آینده رادر تهران برگزار می‌کنیم ، اما دسته‌بندی حوزهها برای رای‌گیری از این هفته آغاز‌شده‌است

['توانتشارخبرمهم', 'در', 'رسانه', 'ها', 'را', 'تایید', 'کردی']
توانتشارخبرمهم در رسانه‌ها را تایید کردی

['پسردکتراحمدی', 'درکارنامه', 'حرفه', 'ای', 'خود', '۱۸', 'سریال', 'تلویزیونی', 'و', '۱۲', 'فیلم', 'سینمایی', 'دارد']
پسردکتراحمدی درکارنامه حرفه‌ای خود 18 سریال تلویزیونی و 12 فیلم سینمایی دارد

['نقّاشی', 'ساختمان', 'به', 'پایان', 'رسیدددددددد']
نقّاشی ساختمان به پایان رسیدددددددد

['اللة', 'اکبر', 'گویان', 'وارد', 'مسجد', 'شد']
الله اکبر گویان وارد مسجد شد

['دکتر', 'جان', 'مممممممممممممممنونم', 'که', 'مشکلات', 'حرکتي', 'فرزندم', 'را', 'درمان', 'کردید']
دکتر جان مممممممممممممممنونم که مشکلات حرکتی فرزندم را درمان کردید



In [64]:
print("""  ----------------- Parsivar ---------------------- """,
      '\n','Normaliz then tokenize\n\n')
ps_tokenizer = parsivar.Tokenizer().tokenize_words
ps_normalizer = parsivar.Normalizer().normalize
for s in sent:
  x = ps_tokenizer(ps_normalizer(doc_string = s))
  for tok in x:
    print(tok)
  print('\nNext sentence')

  ----------------- Parsivar ----------------------  
 Normaliz then tokenize


انتخابات
هفته
آینده
رادر
تهران
برگزار
می‌کنیم
،
اما
دسته‌بندی
حوزهها
برای
رای‌گیری
از
این
هفته
آغاز‌شده‌است

Next sentence
توانتشارخبرمهم
در
رسانه‌ها
را
تایید
کردی

Next sentence
پسردکتراحمدی
درکارنامه
حرفه‌ای
خود
18
سریال
تلویزیونی
و
12
فیلم
سینمایی
دارد

Next sentence
نقّاشی
ساختمان
به
پایان
رسیدددددددد

Next sentence
الله
اکبر
گویان
وارد
مسجد
شد

Next sentence
دکتر
جان
مممممممممممممممنونم
که
مشکلات
حرکتی
فرزندم
را
درمان
کردید

Next sentence


In [65]:
"""  ----------------- Dadmatools ---------------------- """


"""              *** IMPORTANT NOTE ***: 
I have run this cell on my local machine.
Since Colab cannot import PROTOCOL_TLS I used my own system.
PROTOCOL_TLS is available for py 3.8> but Colab uses py 3.7
"""


# !pip install dadmatools 
# import dadmatools

# sent = [
# 'انتخابات هفته آینده رادر تهران برگزار میکنیم ، اما دسته بندی حوزهها برای رأی گیری از این هفته آغاز شده است',
# 'توانتشارخبرمهم در رسانه ها را تایید کردی',
# 'پسردکتراحمدی درکارنامه حرفه ای خود ۱۸ سریال تلویزیونی و ۱۲ فیلم سینمایی دارد',
# 'نقّاشی ساختمان به پایان رسیدددددددد',
# 'اللة اکبر گویان وارد مسجد شد',
# 'دکتر جان مممممممممممممممنونم که مشکلات حرکتي فرزندم را درمان کردید',
# ]

# from dadmatools.models.normalizer import Normalizer

# normalizer = Normalizer(
#     full_cleaning=False,
#     unify_chars=True,
#     refine_punc_spacing=True,
#     remove_extra_space=True,
#     remove_puncs=False,
#     remove_html=False,
#     remove_stop_word=False,
#     replace_email_with="<EMAIL>",
#     replace_number_with=None,
#     replace_url_with="",
#     replace_mobile_number_with=None,
#     replace_emoji_with=None,
#     replace_home_number_with=None
# )

# dm_normalized_lst = []
# for s in sent:
#     dm_normalized_lst.append(normalizer.normalize(s))
# # dm_normalized_lst

# from  dadmatools.pipeline import language 
# # here lemmatizer and pos tagger will be loaded
# # as tokenizer is the default tool, it will be loaded as well even without calling
# pips = 'tok,lem' 
# nlp = language.Pipeline(pips)

# # you can see the pipeline with this code
# print(nlp.analyze_pipes(pretty=True))


# dm_final_result = []
# for d in dm_docs:
#     dm_final_result.append(language.to_json(pips, d))

# dm_final_result

# import pickle
# with open('dm_normalized_lst.pkl', 'wb') as f:
#     pickle.dump(dm_normalized_lst, f)

# import pickle
# with open('dm_final_result.pkl', 'wb') as f:
#     pickle.dump(dm_final_result, f)

'              *** IMPORTANT NOTE ***: \nI have run this cell on my local machine.\nSince Colab cannot import PROTOCOL_TLS I used my own system.\nPROTOCOL_TLS is available for py 3.8> but Colab uses py 3.7\n'

In [66]:
import pickle
with open('dm_normalized_lst.pkl', 'rb') as f:
  dm_normalized = pickle.load(f)

dm_normalized

['انتخابات هفته آینده رادر تهران برگزار میکنیم، اما دسته بندی حوزهها برای رای گیری از این هفته آغاز شده است',
 'توانتشارخبرمهم در رسانه ها را تایید کردی',
 'پسردکتراحمدی درکارنامه حرفه ای خود 18 سریال تلویزیونی و 12 فیلم سینمایی دارد',
 'نقّاشی ساختمان به پایان رسیدددددددد',
 'الله اکبر گویان وارد مسجد شد',
 'دکتر جان مممممممممممممممنونم که مشکلات حرکتی فرزندم را درمان کردید']

### Part one Corrolary: 
1) Case 1: 
""" آغاز شده است"""

Hazm: شده_است

parsivar: آغازشده‌است

dadma: آغاز شده است

Unexpectedly, Parsivar has outperformed other libraries in normalizing compound verbs in persian.

2) Case 2:  " رای "

both Parsivar and dadma have done a pretty good job in removing "ء". Yet, all libraries have a weakness in this task. For instance non of them has removed " Tashdid" from " نقّاشی " 

3) Case 3 : "مممممممنونم"

Needless to say, all of them have failed to normalize this case and many others.

3) Case 3 : "رادر"

Needless to say, all of them have failed to tokenize this case and many others.





## Q2 (Stemming/Lemma, StopWords Removal)





In [67]:
"""  ----------------- Hazm ---------------------- """
hz_stops = hazm.stopwords_list()
hz_stemmer  = hazm.Stemmer().stem
hz_lemmatizer = hazm.Lemmatizer().lemmatize

for s in sent:
  toks = s.strip().split()
  stems = [hz_stemmer(t) for t in toks]
  lems = [hz_lemmatizer(t) for t in toks]
  stp = [tok for tok in toks if tok in hz_stops ]
  print(f'List of stopwrods:{stp}')
  print(f'stemming: {stems}')
  print(f'lemming: {lems}')
  print('\n\n')



List of stopwrods:['اما', 'دسته', 'بندی', 'برای', 'گیری', 'از', 'این', 'شده', 'است']
stemming: ['انتخاب', 'هفته', 'آینده', 'رادر', 'تهر', 'برگزار', 'میکن', '،', 'اما', 'دسته', 'بند', 'حوزه', 'برا', 'رأ', 'گیر', 'از', 'این', 'هفته', 'آغاز', 'شده', 'اس']
lemming: ['انتخابات', 'هفته', 'آینده', 'رادر', 'تهران', 'برگزار', 'میکنیم', '،', 'اما', 'دسته', 'بست#بند', 'حوزه', 'برای', 'رأی', 'گرفت#گیر', 'از', 'این', 'هفته', 'آغاز', 'شده', '#است']



List of stopwrods:['در', 'را']
stemming: ['توانتشارخبرمه', 'در', 'رسانه', '', 'را', 'تایید', 'کرد']
lemming: ['توانتشارخبرمهم', 'در', 'رسانه', 'ها', 'را', 'تایید', 'کردی']



List of stopwrods:['خود', 'و', 'دارد']
stemming: ['پسردکتراحمد', 'درکارنامه', 'حرفه', 'ا', 'خود', '۱۸', 'سریال', 'تلویزیون', 'و', '۱۲', 'فیل', 'سینما', 'دارد']
lemming: ['پسردکتراحمدی', 'درکارنامه', 'حرفه', 'ای', 'خود', '۱۸', 'سریال', 'تلویزیون', 'و', '۱۲', 'فیلم', 'سینمایی', 'داشت#دار']



List of stopwrods:['به']
stemming: ['نقّاش', 'ساخ', 'به', 'پا', 'رسیدددددددد']
lemming: ['ن

In [68]:
"""  ----------------- Parsivar ---------------------- """

''' Note1: Parsivar only support following functionalities '''
''' Note2: Parsivar's Stemmer Doesn't have any interface'''

"""
functionalities:
          Text Normalizing
          Half space correction in Persian text
          Word and sentence tokenizer (splitting words and sentences)
          Word stemming
          POS tagger
          Shallow parser (Chunker)
          Dependency Parser
          Spell Checker
"""


'\nfunctionalities:\n          Text Normalizing\n          Half space correction in Persian text\n          Word and sentence tokenizer (splitting words and sentences)\n          Word stemming\n          POS tagger\n          Shallow parser (Chunker)\n          Dependency Parser\n          Spell Checker\n'

In [69]:
"""  ----------------- Dadmatools ---------------------- """
'''Note: Dadmatools Doesn't remove stopword, only lemmetization
You can see its lemmetization result in the next part'''


"Note: Dadmatools Doesn't remove stopword, only lemmetization\nYou can see its lemmetization result in the next part"

### Part two Corrolary: 
1) Case 1: 
""" فیلم >>>‌  فیل""


Hazm: hazm stemmes " Film " into " Fil " that might cuase ambiguity in some specific tasks, like categorization.


More than what I have already described, there is no discernible effect or behaviour.



## Q3 (Question 2 and 3 consecutively)

In [70]:
"""  ----------------- Hazm ---------------------- """
# hz_tokenizer = hazm.word_tokenize
# hz_normalizer = hazm.Normalizer().normalize
# for s in sent:
#   print(hz_tokenizer(s))
#   print(hz_normalizer(s))
#   print()
# hz_stops = hazm.stopwords_list()
# hz_stemmer  = hazm.Stemmer().stem
# hz_lemmatizer = hazm.Lemmatizer().lemmatize

for s in sent:
  s = hz_normalizer(s)
  toks = hz_tokenizer(s)
  stems = [hz_stemmer(t) for t in toks]
  lems = [hz_lemmatizer(t) for t in toks]
  stp = [tok for tok in toks if tok in hz_stops ]
  print(f'List of stopwrods:{stp}')
  print(f'Out Put of HAZM pipeline after stemming : {stems}')
  print(f'Out Put of HAZM pipeline after lemming  : {lems}')
  print('\n\n')

List of stopwrods:['اما', 'دسته', 'بندی', 'برای', 'گیری', 'از', 'این', 'شده_است']
Out Put of HAZM pipeline after stemming : ['انتخاب', 'هفته', 'آینده', 'رادر', 'تهر', 'برگزار', 'میکن', '،', 'اما', 'دسته', 'بند', 'حوزه', 'برا', 'رأ', 'گیر', 'از', 'این', 'هفته', 'آغاز', 'شده_اس']
Out Put of HAZM pipeline after lemming  : ['انتخابات', 'هفته', 'آینده', 'رادر', 'تهران', 'برگزار', 'میکنیم', '،', 'اما', 'دسته', 'بست#بند', 'حوزه', 'برای', 'رأی', 'گرفت#گیر', 'از', 'این', 'هفته', 'آغاز', 'شد#شو']



List of stopwrods:['در', 'را']
Out Put of HAZM pipeline after stemming : ['توانتشارخبرمه', 'در', 'رسانه', 'را', 'تایید', 'کرد']
Out Put of HAZM pipeline after lemming  : ['توانتشارخبرمهم', 'در', 'رسانه', 'را', 'تایید', 'کردی']



List of stopwrods:['خود', 'و', 'دارد']
Out Put of HAZM pipeline after stemming : ['پسردکتراحمد', 'درکارنامه', 'حرفه', 'خود', '۱۸', 'سریال', 'تلویزیون', 'و', '۱۲', 'فیل', 'سینما', 'دارد']
Out Put of HAZM pipeline after lemming  : ['پسردکتراحمدی', 'درکارنامه', 'حرفه\u200cای', 

In [71]:
"""  ----------------- Parsivar ---------------------- """
''' Not applicable '''

' Not applicable '

In [72]:
"""  ----------------- Dadmatools ---------------------- """
import pickle
with open('dm_final_result.pkl', 'rb') as f:
  dm_final_res = pickle.load(f)
  
dm_final_res

[[[{'id': 1, 'lemma': 'انتخابات', 'text': 'انتخابات'},
   {'id': 2, 'lemma': 'هفته', 'text': 'هفته'},
   {'id': 3, 'lemma': 'آینده', 'text': 'آینده'},
   {'id': 4, 'lemma': 'رادر', 'text': 'رادر'},
   {'id': 5, 'lemma': 'تهران', 'text': 'تهران'},
   {'id': 6, 'lemma': 'برگزار', 'text': 'برگزار'},
   {'id': 7, 'lemma': 'میکنیم', 'text': 'میکنیم'},
   {'id': 8, 'lemma': '،', 'text': '،'},
   {'id': 9, 'lemma': 'اما', 'text': 'اما'},
   {'id': 10, 'lemma': 'دسته', 'text': 'دسته'},
   {'id': 11, 'lemma': 'بند', 'text': 'بندی'},
   {'id': 12, 'lemma': 'حوزه', 'text': 'حوزهها'},
   {'id': 13, 'lemma': 'برای', 'text': 'برای'},
   {'id': 14, 'lemma': 'رای', 'text': 'رای'},
   {'id': 15, 'lemma': 'گیر', 'text': 'گیری'},
   {'id': 16, 'lemma': 'از', 'text': 'از'},
   {'id': 17, 'lemma': 'این', 'text': 'این'},
   {'id': 18, 'lemma': 'هفته', 'text': 'هفته'},
   {'id': 19, 'lemma': 'آغاز', 'text': 'آغاز'},
   {'id': 20, 'lemma': 'شد#شو', 'text': 'شده'},
   {'id': 21, 'lemma': '#است', 'text': 'است'}

### Part three Corrolary: 

1) Case 1: dadma seems to do the same job in lemmetization as Hazm. Nothing more.


2) Case 2:  " شده است "

Lemmitization on init text results in: 'شده', '#است

However, Lemmitization on nomalized text results in "شد#شو"



3) Case 3) BE ADVISED! IF YOU STEMM THEN REMOVE STOP WORDS, YOUR PREPROCESSING RESULT IN A BIASED TEXT.

FOR INSTANCE, "در" is a stop word. درمان will be stemmed into 'در' by hazm lib. So if you do stemming and then stop word removal, you remove some important which may have discrimination power!


## Q4 (Libraries Comparision)



1.   Hazm: 
      
     hazm supports wide range of functions for preprocessing.
     Since it mostly uses Regular Expression for its preprocessing does not perform well in Normalization. Lastly, Hazm performs fine on lemmetization of verbs.  
2.   Parsivar:

 Parsivar has a limited range of preprocessing funcitons. Does not have stop word list. Its stemming module does not have an interface and if you wanna use its stemming module, you gotta write an interface for it. like hazm, parsivar didn't normalize "مممممنون" or "پسردکتراحمدی" or others. 

3.   Dadmatools: unlike hazm and Parsivar, dadma uses new approaches like neural networks and transformers to reach higher performance on Persian NLP tasks. It, however, does not use new approaches for preprocessing (normalizing , tokenization, and lemmitization). That's why dadma does not outperform hazm on prerprocesing task.

All in all, I prefer using Hazm for preprocessing tasks and Dadam for other Persian NLP tasks.



### Persian NLP Datasets That Dadmatools supports

We provide an easy-to-use way to load some popular Persian NLP datasets
Here is the list of supported datasets.

   |    Dataset             | Task 
|       :----------------:               |  :----------------:   
   |    PersianNER           |   Named Entity Recognition   | 
   |       ARMAN             |   Named Entity Recognition
   |       Peyma             | Named Entity Recognition
  |       FarsTail           | Textual Entailment
 |        FaSpell           | Spell Checking
  |      PersianNews        | Text Classification
  |       PerUDT            | Universal Dependency
  |      PnSummary          | Text Summarization
  |    SnappfoodSentiment   | Sentiment Classification
  |           TEP           | Text Translation(eng-fa)
| WikipediaCorpus               | Corpus
| PersianTweets           | Corpus




### Evaluation Dadma VS Hamz
Dadmatools have compared their pos tagging, dependancy parsing, and lemmatization models to `stanza` and `hazm`.

<table>
  <tr align='center'>
    <td colspan="4"><b>PerDT (F1 score)</b></td>
  </tr>
  <tr align='center'>
    <td><b>Toolkit</b></td>
    <td><b>POS Tagger (UPOS)</b></td>
    <td><b>Dependancy Parser (UAS/LAS)</b></td>
    <td><b>Lemmatizer</b></td>
  </tr>
  <tr align='center'>
    <td>DadmaTools</td>
    <td><b>97.52%</b></td>
    <td><b>95.36%</b>  /  <b>92.54%</b> </td>
    <td><b>99.14%</b> </td>
  </tr>
  <tr align='center'>
    <td>stanza</td>
    <td>97.35%</td>
    <td>93.34%  /  91.05% </td>
    <td>98.97% </td>
  </tr>
  <tr align='center'>
    <td>hazm</td>
    <td>-</td>
    <td>- </td>
    <td>89.01% </td>
  </tr>


  <tr align='center'>
    <td colspan="4"><b>Seraji (F1 score)</b></td>
  </tr>
  <tr align='center'>
    <td><b>Toolkit</b></td>
    <td><b>POS Tagger (UPOS)</b></td>
    <td><b>Dependancy Parser (UAS/LAS)</b></td>
    <td><b>Lemmatizer</b></td>
  </tr>
  <tr align='center'>
    <td>DadmaTools</td>
    <td><b>97.83%</b></td>
    <td><b>92.5%</b>  /  <b>89.23%</b> </td>
    <td> - </td>
  </tr>
  <tr align='center'>
    <td>stanza</td>
    <td>97.43%</td>
    <td>87.20% /  83.89% </td>
    <td> - </td>
  </tr>
  <tr align='center'>
    <td>hazm</td>
    <td>-</td>
    <td>- </td>
    <td>86.93% </td>
  </tr>
</table>


<table>
  <tr align='center'>
    <td colspan="2"><b>Tehran university tree bank (F1 score)</b></td>
  </tr>
  <tr align='center'>
    <td><b>Toolkit</b></td>
    <td><b>Constituency Parser</b></td>
  </tr>
  <tr align='center'>
    <td>DadmaTools (without preprocess))</td>
    <td><b>82.88%</b></td>
  </tr>
  <tr align='center'>
    <td>Stanford (with some preprocess on POS tags)</td>
    <td>80.28</td>
  </tr>
</table>

Reference: Dadmatools github
https://github.com/Dadmatech/DadmaTools/edit/main/README.md

## Q5 (levenshtein distance) 

In [31]:
!gdown --id 1rovazK48q7pHcEM271aX70Dr594NYQ77
!gdown --id 1ZCHuj6JtyOkb5ismn3qF3Rp2DRGJ1tRk
cls()

In [32]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
tmp1 = train_df['article'].tolist() 
all = tmp1 + test_df['article'].tolist()

parsi = FarsiParsi()
all_cooked = [parsi.clean_tokenize(x) for x in all]
d = {}
for lst in all_cooked:
  for tok in lst:
    if tok in d:
        d[tok] += 1
    else:
        d[tok] = 1
vocab_lst = []
for k,v in d.items():
  if v>3:
    vocab_lst.append(k)

from random import sample
print(sample(vocab_lst,30))

['فری', 'توپ', 'درجا', 'قوام', 'کاتای', 'هماهنگ', 'غضروف', 'فلوریدای', 'قدمتی', 'توجیهی', 'جلوه', 'ادامه', 'اصالت', 'اتفاقیه', 'المانهای', 'کشکان', 'غشاأ', 'تهرانپارس', 'فرضیات', 'بازنگری', 'ملکوت', 'جواد', 'حالت', 'کلیث', 'موقتی', 'تار', 'عادات', 'شیای', 'تعبیرات', 'واحدی']


In [33]:
chars = {}
for tok in vocab_lst:
  for char in tok:
    if not char in chars:
        chars[char] = 1

"""
More info about ASCII codes:
https://miro.medium.com/max/1200/1*DdgD00dAdXggzMdWDt7GSA.png
"""
x = [a for a in 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLM'] 

per_eng_dict = {}
i = -1
for k in  chars.keys():
  i+=1 
  per_eng_dict[k] = x[i]

# >> per_eng_dict
# {'_': 'G',
#  'آ': 'C',
#  'أ': 'J',
#  'ؤ': 'H',
#  'إ': 'K',
#  'ا': 'e',
#  'ب': 'n',
#  'ت': 'm',
#  'ث': 'A',
#  'ج': 'u',
#  'ح': 'o',
#  'خ': 'v',
#  . . .

### Weighted MINIMUM EDIT DISTANCE WITH LIB



In [43]:
spell = SpellCorrection()
spell.fit(per_eng_dict,vocab_lst)
correction_candidates = []
for w in words:
  print(f'Based on Min Edit Distance, bests for {w} are:')
  tmp = spell.predict(string = w , top_bests = 5 )
  correction_candidates.append(tmp)
  print(tmp,'\n')

candidate_words = []
for sett in correction_candidates:
  tmp = []
  for can in sett:
    tmp.append(can[1])
  candidate_words.append(tmp)
candidate_words

Based on Min Edit Distance, bests for اختصاد are:
[[2.0, 'اختصاص'], [2.0, 'اقتصاد'], [2.0, 'اختصار'], [3.0, 'اقتصادی'], [3.0, 'افتاد']] 

Based on Min Edit Distance, bests for سادرات are:
[[1.0, 'سادات'], [2.0, 'خسارات'], [2.0, 'صادرات'], [2.0, 'سارا'], [2.0, 'ادات']] 

Based on Min Edit Distance, bests for فوتکال are:
[[2.0, 'فوتبال'], [2.0, 'فوتسال'], [3.0, 'فوت'], [3.0, 'فواصل'], [3.0, 'تکامل']] 

Based on Min Edit Distance, bests for مسابغاط are:
[[3.0, 'سابا'], [4.0, 'مساال'], [4.0, 'مسابقات'], [4.0, 'محابا'], [4.0, 'سابقا']] 

Based on Min Edit Distance, bests for وازدات are:
[[2.0, 'واداشت'], [2.0, 'موازات'], [2.0, 'واردات'], [2.0, 'ادات'], [3.0, 'وزارت']] 

Based on Min Edit Distance, bests for مشرکت are:
[[1.0, 'شرکت'], [1.0, 'مشارکت'], [1.0, 'مشرک'], [2.0, 'مشترک'], [2.0, 'شرکتی']] 

Based on Min Edit Distance, bests for کشوور are:
[[1.0, 'کشور'], [2.0, 'شور'], [2.0, 'کشوری'], [2.0, 'کشورش'], [2.0, 'کور']] 

Based on Min Edit Distance, bests for منجلسه are:
[[2.0, 'مجلس'], [2

[['اختصاص', 'اقتصاد', 'اختصار', 'اقتصادی', 'افتاد'],
 ['سادات', 'خسارات', 'صادرات', 'سارا', 'ادات'],
 ['فوتبال', 'فوتسال', 'فوت', 'فواصل', 'تکامل'],
 ['سابا', 'مساال', 'مسابقات', 'محابا', 'سابقا'],
 ['واداشت', 'موازات', 'واردات', 'ادات', 'وزارت'],
 ['شرکت', 'مشارکت', 'مشرک', 'مشترک', 'شرکتی'],
 ['کشور', 'شور', 'کشوری', 'کشورش', 'کور'],
 ['مجلس', 'جلسه', 'مجله', 'منجمله', 'مجسمه']]

### Weighted MINIMUM EDIT DISTANCE FROM SCRACH ( WITHOUT ANY LIB )

In [44]:
correction_candidates = []
for w in words:
  print(f'From Scratch!, bests for {w} are:')
  tmp = spell.predict_from_scratch(string = w , top_bests = 5 )
  correction_candidates.append(tmp)
  print(tmp,'\n')

candidate_words = []
for sett in correction_candidates:
  tmp = []
  for can in sett:
    tmp.append(can[1])
  candidate_words.append(tmp)
candidate_words

From Scratch!, bests for اختصاد are:
[[2, 'اختصاص'], [2, 'اقتصاد'], [2, 'اختصار'], [3, 'اقتصادی'], [3, 'افتاد']] 

From Scratch!, bests for سادرات are:
[[1, 'سادات'], [2, 'خسارات'], [2, 'صادرات'], [2, 'سارا'], [2, 'ادات']] 

From Scratch!, bests for فوتکال are:
[[2, 'فوتبال'], [2, 'فوتسال'], [3, 'فوت'], [3, 'فواصل'], [3, 'تکامل']] 

From Scratch!, bests for مسابغاط are:
[[3, 'سابا'], [4, 'مساال'], [4, 'مسابقات'], [4, 'محابا'], [4, 'سابقا']] 

From Scratch!, bests for وازدات are:
[[2, 'واداشت'], [2, 'موازات'], [2, 'واردات'], [2, 'ادات'], [3, 'وزارت']] 

From Scratch!, bests for مشرکت are:
[[1, 'شرکت'], [1, 'مشارکت'], [1, 'مشرک'], [2, 'مشترک'], [2, 'شرکتی']] 

From Scratch!, bests for کشوور are:
[[1, 'کشور'], [2, 'شور'], [2, 'کشوری'], [2, 'کشورش'], [2, 'کور']] 

From Scratch!, bests for منجلسه are:
[[2, 'مجلس'], [2, 'جلسه'], [2, 'مجله'], [2, 'منجمله'], [3, 'مجسمه']] 



[['اختصاص', 'اقتصاد', 'اختصار', 'اقتصادی', 'افتاد'],
 ['سادات', 'خسارات', 'صادرات', 'سارا', 'ادات'],
 ['فوتبال', 'فوتسال', 'فوت', 'فواصل', 'تکامل'],
 ['سابا', 'مساال', 'مسابقات', 'محابا', 'سابقا'],
 ['واداشت', 'موازات', 'واردات', 'ادات', 'وزارت'],
 ['شرکت', 'مشارکت', 'مشرک', 'مشترک', 'شرکتی'],
 ['کشور', 'شور', 'کشوری', 'کشورش', 'کور'],
 ['مجلس', 'جلسه', 'مجله', 'منجمله', 'مجسمه']]

## Q6 (Incorporate Bigrams)

In [ ]:
accum_bigrams = accum_ngram([parsi.clean_tokenize(x) for x in all], n_gram=2)

In [ ]:
question6_handler(candidate_words,bigrams_sent,accum_bigrams,mu = 0.1,Pbg = (1/spell.vocab_len))


Based on Bigrams, best word:اقتصادی witg prob:2.3056409667754882e-11

Based on Bigrams, best word:صادرات witg prob:1.390483839012269e-07

Based on Bigrams, best word:فوتبال witg prob:2.0465085532346898e-06

Based on Bigrams, best word:مسابقات witg prob:7.464697729422069e-11

Based on Bigrams, best word:واردات witg prob:2.8311229279674023e-10

Based on Bigrams, best word:مشارکت witg prob:6.989758056699232e-08

Based on Bigrams, best word:کشور witg prob:2.1509679954252117e-08

Based on Bigrams, best word:مجلس witg prob:1.7246005873001862e-08



# Further Developments notes:

Better DS for "predict" function from "SpellCorrection" class

In [ ]:
whos

Variable                         Type                          Data/Info
------------------------------------------------------------------------
FarsiParsi                       type                          <class '__main__.FarsiParsi'>
SpellCorrection                  type                          <class '__main__.SpellCorrection'>
accum_bigrams                    dict                          n=52485
accum_ngram                      function                      <function accum_ngram at 0x7f5e3ece0e60>
all                              list                          n=814
all_cooked                       list                          n=814
bigrams_sent                     list                          n=8
can                              list                          n=2
candidate_words                  list                          n=8
candidates                       list                          n=8
char                             str                           و
chars            